### Importando dependências

In [1]:
import numpy as np
import pandas as pd

## Importação dos dados


In [2]:
dicionario = pd.read_csv('../datasets/dicionario_nascidos_vivos.csv')
df = pd.read_csv('../datasets/nascidos_vivos.csv')

In [3]:
df.head()

,Unnamed: 0,ano,sigla_uf,local_nascimento,data_nascimento,hora_nascimento,sexo,peso,raca_cor,id_anomalia,...,id_uf_mae,idade_mae,escolaridade_mae,estado_civil_mae,ocupacao_mae,raca_cor_mae,gestacoes_ant,quantidade_parto_normal,quantidade_parto_cesareo,idade_pai
0,0,2016,MT,1,2016-01-25,12:20:00,2,4235.0,1.0,2.0,...,50,37.0,4.0,5.0,NaN,1.0,3.0,1.0,2.0,NaN
1,1,2016,MT,1,2016-02-01,11:21:00,2,3145.0,1.0,2.0,...,51,27.0,4.0,5.0,NaN,1.0,2.0,0.0,2.0,NaN
2,2,2016,MT,1,2016-02-07,00:44:00,2,3315.0,4.0,2.0,...,51,25.0,4.0,2.0,NaN,4.0,0.0,0.0,0.0,NaN
3,3,2016,MT,1,2016-04-11,14:05:00,1,3275.0,4.0,2.0,...,51,27.0,4.0,2.0,NaN,4.0,3.0,3.0,0.0,NaN
4,4,2016,MT,1,2016-04-15,23:40:00,2,3500.0,2.0,2.0,...,26,16.0,4.0,5.0,NaN,2.0,1.0,1.0,0.0,21.0


In [4]:
dicionario.head()

,id_tabela,coluna,chave,cobertura_temporal,valor
0,microdados,local_nascimento,1,(1),Hospital
1,microdados,local_nascimento,2,(1),Outros estabelecimentos de saúde
2,microdados,local_nascimento,3,(1),Domicílio
3,microdados,local_nascimento,4,(1),Outros
4,microdados,local_nascimento,9,(1),Ignorado


## Tratando dados de 2021

Os dados de 2021 não estavam presentes na base de dados original. No entanto, eles são muito relevantes para a nossa análise pois representam com maior precisão o período pandêmico. Sendo assim, realizamos o download do arquivo separadamente no site [openDataSUS](https://opendatasus.saude.gov.br/dataset/sistema-de-informacao-sobre-nascidos-vivos-sinasc).


In [5]:
df_2021 = pd.read_csv('../datasets/nascidos_vivos_2021.csv', delimiter=';')
df_2021.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,TPDOCRESP,DTDECLARAC,ESCMAEAGR1,STDNEPIDEM,STDNNOVA,CODPAISRES,TPROBSON,PARIDADE,KOTELCHUCK,CONTADOR
0,1,2515768.0,110010,1,18.0,1.0,4.0,NaN,0.0,1.0,...,3.0,6042021.0,5.0,0.0,1,1.0,3,1,3,2815
1,1,2516276.0,110015,1,28.0,2.0,5.0,NaN,0.0,0.0,...,3.0,5022021.0,8.0,0.0,1,1.0,8,0,3,4951
2,1,2496879.0,110015,1,32.0,5.0,4.0,999992.0,1.0,0.0,...,5.0,11062021.0,6.0,0.0,1,1.0,10,1,2,5378
3,1,5618347.0,110020,1,25.0,NaN,3.0,999992.0,4.0,1.0,...,4.0,2042021.0,11.0,0.0,1,1.0,3,1,9,7784
4,1,5618347.0,110020,1,32.0,2.0,5.0,999992.0,1.0,2.0,...,3.0,28062021.0,7.0,0.0,1,1.0,5,1,9,8425


### 1. Descartando colunas

In [6]:
columns_to_drop = ['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'CODMUNRES', 'APGAR1', 'APGAR5', 'DTCADASTRO', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA', 'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'ESCMAE2010', 'SERIESCMAE', 'DTNASCMAE', 'DTULTMENST', 'TPMETESTIM', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO', 'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1', 'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE', 'KOTELCHUCK', 'CONTADOR', 'SEMAGESTAC', 'MESPRENAT']

df_2021.drop(columns_to_drop, axis=1, inplace=True)

In [7]:
df_2021.columns

Index(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'CODOCUPMAE',
       'QTDFILVIVO', 'QTDFILMORT', 'GESTACAO', 'GRAVIDEZ', 'PARTO',
       'CONSULTAS', 'DTNASC', 'HORANASC', 'SEXO', 'RACACOR', 'PESO',
       'IDANOMAL', 'CODANOMAL', 'CODUFNATU', 'RACACORMAE', 'QTDGESTANT',
       'QTDPARTNOR', 'QTDPARTCES', 'IDADEPAI', 'CONSPRENAT'],
      dtype='object')

In [8]:
df.columns

Index(['Unnamed: 0', 'ano', 'sigla_uf', 'local_nascimento', 'data_nascimento',
       'hora_nascimento', 'sexo', 'peso', 'raca_cor', 'id_anomalia',
       'codigo_anomalia', 'semana_gestacao', 'semana_gestacao_estimada',
       'gestacao_agr', 'tipo_gravidez', 'tipo_parto', 'inicio_pre_natal',
       'pre_natal', 'pre_natal_agr', 'classificacao_pre_natal',
       'quantidade_filhos_vivos', 'quantidade_filhos_mortos', 'id_uf_mae',
       'idade_mae', 'escolaridade_mae', 'estado_civil_mae', 'ocupacao_mae',
       'raca_cor_mae', 'gestacoes_ant', 'quantidade_parto_normal',
       'quantidade_parto_cesareo', 'idade_pai'],
      dtype='object')

### 2. Renomeando colunas

In [9]:
df_2021['ano'] = 2021

rename_map = {
    'CODUFNATU': 'sigla_uf',
    'LOCNASC': 'local_nascimento', 
    'DTNASC': 'data_nascimento',
    'HORANASC': 'hora_nascimento',
    'SEXO': 'sexo',
    'PESO': 'peso',
    'RACACOR': 'raca_cor',
    'IDANOMAL': 'id_anomalia',
    'CODANOMAL': 'codigo_anomalia',
    'SEMANAGESTAC': 'semana_gestacao',
    'GESTACAO': 'gestacao_agr',
    'GRAVIDEZ': 'tipo_gravidez',
    'PARTO': 'tipo_parto',
    'CONSPRENAT': 'pre_natal',
    'CONSULTAS': 'pre_natal_agr',
    'QTDFILVIVO': 'quantidade_filhos_vivos',
    'QTDFILMORT': 'quantidade_filhos_mortos',
    'IDADEMAE': 'idade_mae',
    'ESCMAE': 'escolaridade_mae',
    'ESTCIVMAE': 'estado_civil_mae',
    'CODOCUPMAE': 'ocupacao_mae',
    'RACACORMAE': 'raca_cor_mae',
    'QTDGESTANT': 'gestacoes_ant',
    'QTDPARTNOR': 'quantidade_parto_normal',
    'QTDPARTCES': 'quantidade_parto_cesareo',
    'IDADEPAI': 'idade_pai'
}

df_2021 = df_2021.rename(columns=rename_map)


In [10]:
df_2021.columns

Index(['local_nascimento', 'idade_mae', 'estado_civil_mae', 'escolaridade_mae',
       'ocupacao_mae', 'quantidade_filhos_vivos', 'quantidade_filhos_mortos',
       'gestacao_agr', 'tipo_gravidez', 'tipo_parto', 'pre_natal_agr',
       'data_nascimento', 'hora_nascimento', 'sexo', 'raca_cor', 'peso',
       'id_anomalia', 'codigo_anomalia', 'sigla_uf', 'raca_cor_mae',
       'gestacoes_ant', 'quantidade_parto_normal', 'quantidade_parto_cesareo',
       'idade_pai', 'pre_natal', 'ano'],
      dtype='object')

### 3. Juntando dados

In [11]:
df = pd.concat([df, df_2021], axis=0)
df

,Unnamed: 0,ano,sigla_uf,local_nascimento,data_nascimento,hora_nascimento,sexo,peso,raca_cor,id_anomalia,...,id_uf_mae,idade_mae,escolaridade_mae,estado_civil_mae,ocupacao_mae,raca_cor_mae,gestacoes_ant,quantidade_parto_normal,quantidade_parto_cesareo,idade_pai
0,0.0,2016,MT,1,2016-01-25,12:20:00,2,4235.0,1.0,2.0,...,50,37.0,4.0,5.0,NaN,1.0,3.0,1.0,2.0,NaN
1,1.0,2016,MT,1,2016-02-01,11:21:00,2,3145.0,1.0,2.0,...,51,27.0,4.0,5.0,NaN,1.0,2.0,0.0,2.0,NaN
2,2.0,2016,MT,1,2016-02-07,00:44:00,2,3315.0,4.0,2.0,...,51,25.0,4.0,2.0,NaN,4.0,0.0,0.0,0.0,NaN
3,3.0,2016,MT,1,2016-04-11,14:05:00,1,3275.0,4.0,2.0,...,51,27.0,4.0,2.0,NaN,4.0,3.0,3.0,0.0,NaN
4,4.0,2016,MT,1,2016-04-15,23:40:00,2,3500.0,2.0,2.0,...,26,16.0,4.0,5.0,NaN,2.0,1.0,1.0,0.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2677096,NaN,2021,17,3,19092021,2313.0,2,3190.0,1.0,2.0,...,NaN,33.0,5.0,2.0,223115.0,1.0,1.0,1.0,0.0,37.0
2677097,NaN,2021,17,1,30102021,319.0,2,3360.0,4.0,9.0,...,NaN,27.0,3.0,1.0,999992.0,4.0,2.0,2.0,0.0,NaN
2677098,NaN,2021,17,1,21092021,834.0,1,2575.0,2.0,2.0,...,NaN,26.0,5.0,2.0,212415.0,2.0,0.0,0.0,0.0,25.0
2677099,NaN,2021,29,1,1122021,730.0,1,3380.0,NaN,2.0,...,NaN,36.0,4.0,5.0,999992.0,NaN,3.0,2.0,0.0,NaN


In [12]:
uf = df['sigla_uf'].astype(str).str.replace('.0', '').unique()
uf.sort()
print(uf)

['11' '12' '13' '14' '15' '16' '17' '21' '22' '23' '24' '25' '26' '27'
 '28' '29' '31' '32' '33' '35' '41' '42' '43' '50' '51' '52' '53' '99'
 'AC' 'AL' 'AM' 'AP' 'BA' 'CE' 'DF' 'ES' 'GO' 'MA' 'MG' 'MS' 'MT' 'PA'
 'PB' 'PE' 'PI' 'PR' 'RJ' 'RN' 'RO' 'RR' 'RS' 'SC' 'SE' 'SP' 'TO' 'nan'
 'nu']


In [13]:
codigo_para_uf = {
    '11': 'RO',
    '12': 'AC',
    '13': 'AM',
    '14': 'RR',
    '15': 'PA',
    '16': 'AP',
    '17': 'TO',
    '21': 'MA',
    '22': 'PI',
    '23': 'CE',
    '24': 'RN',
    '25': 'PB',
    '26': 'PE',
    '27': 'AL',
    '28': 'SE',
    '29': 'BA',
    '31': 'MG',
    '32': 'ES',
    '33': 'RJ',
    '35': 'SP',
    '41': 'PR',
    '42': 'SC',
    '43': 'RS',
    '50': 'MS',
    '51': 'MT',
    '52': 'GO',
    '53': 'DF',
    '99': np.NaN,
    'nu': np.NaN,
    'nan': np.NaN
}

df['sigla_uf'] = df['sigla_uf'] \
    .astype(str) \
    .str.replace('.0', '') \
    .apply(lambda uf: codigo_para_uf[uf] if uf in codigo_para_uf else uf)


In [14]:
df['sigla_uf'].unique()

array(['MT', 'PB', 'ES', 'MS', 'AM', 'GO', 'AP', 'AC', 'RR', 'SC', 'RS',
       'PE', 'CE', 'MA', 'PA', 'SP', 'PR', 'TO', 'RO', 'MG', 'BA', 'RJ',
       'SE', 'DF', 'AL', 'RN', 'PI', nan], dtype=object)

## Limpeza e pré-processamento dos dados


### 1. Descartando colunas irrelevantes

As colunas relevantes já foram filtradas na consulta SQL inicial (Consulte o notebook [Download inicial dos dados]('') para mais detalhes). No entanto, algumas colunas não estão presentes no dataset de 2021. Sendo assim, eliminaremos a coluna indexadora e as que não foram registradas em 2021.

In [15]:
df = df.drop(['Unnamed: 0', 'semana_gestacao_estimada', 'classificacao_pre_natal', 'inicio_pre_natal', 'id_uf_mae', ], axis=1)

### 2. Tratando valores nulos

In [16]:
# Verificando presença de valores nulos
df.isnull().sum()

ano                                0
sigla_uf                       57641
local_nascimento                   0
data_nascimento                    0
hora_nascimento                25670
sexo                               0
peso                            6081
raca_cor                      669113
id_anomalia                   196893
codigo_anomalia             19826866
semana_gestacao              2980799
gestacao_agr                  324805
tipo_gravidez                  24880
tipo_parto                     15380
pre_natal                     395966
pre_natal_agr                   3740
quantidade_filhos_vivos       704558
quantidade_filhos_mortos     1099957
idade_mae                        253
escolaridade_mae              147474
estado_civil_mae              139275
ocupacao_mae                 4465847
raca_cor_mae                  800110
gestacoes_ant                 747709
quantidade_parto_normal      1010093
quantidade_parto_cesareo     1119250
idade_pai                   12135762
d

Nessa etapa evitamos ao máximo a remoção de linhas, já que a análise quantitativa de nascimentos é importante para a análise. Os valores nulos normalmente variam de acordo com a forma de coleta em cada ano. 

Em 2016, por exemplo, o campo `ocupacao_mae` não foi registrado. Remover essas linhas implicaria em remover todos os registros de 2016.

In [17]:
df[df['ano']==2016]['ocupacao_mae'].count()

0

In [18]:
# Campos opcionais ou que não podem ser substituídos
manter_nulo = ['sigla_uf', 'hora_nascimento', 'idade_pai', 'raca_cor', 'codigo_anomalia', 'quantidade_filhos_vivos', 'quantidade_filhos_mortos', 'raca_cor_mae', 'ocupacao_mae', 'gestacoes_ant', 'quantidade_parto_normal', 'quantidade_parto_cesareo']

# Campos numéricos e obrigatórios que não deveriam estar vazios
substituir_pela_media = ['peso', 'semana_gestacao', 'pre_natal', 'idade_mae']

# Campos categóricos opcionais
substituir_por_valor_especifico = ['id_anomalia', 'gestacao_agr', 'tipo_gravidez', 'tipo_parto', 'escolaridade_mae', 'estado_civil_mae']


In [19]:
for column in substituir_por_valor_especifico:
    df[column].fillna(9, inplace=True) # 9 significa 'Ignorado' de acordo com o dicionário

for column in substituir_pela_media:
    df[column].fillna(df[column].mean(), inplace=True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19999765 entries, 0 to 2677100
Data columns (total 27 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   ano                       int64  
 1   sigla_uf                  object 
 2   local_nascimento          int64  
 3   data_nascimento           object 
 4   hora_nascimento           object 
 5   sexo                      int64  
 6   peso                      float64
 7   raca_cor                  float64
 8   id_anomalia               float64
 9   codigo_anomalia           object 
 10  semana_gestacao           float64
 11  gestacao_agr              float64
 12  tipo_gravidez             float64
 13  tipo_parto                float64
 14  pre_natal                 float64
 15  pre_natal_agr             float64
 16  quantidade_filhos_vivos   float64
 17  quantidade_filhos_mortos  float64
 18  idade_mae                 float64
 19  escolaridade_mae          float64
 20  estado_civil_mae          fl

### 3. Removendo linhas duplicadas

In [21]:
df.duplicated().sum()

932

In [22]:
df = df.drop_duplicates()

### 4. Verificando dados inconsistentes

Aqui verificamos se há presença de dados inconsistentes nas principais colunas.

In [23]:
df['ano'].unique()

array([2016, 2019, 2018, 2015, 2017, 2020, 2021])

In [24]:
df['sigla_uf'].unique()

array(['MT', 'PB', 'ES', 'MS', 'AM', 'GO', 'AP', 'AC', 'RR', 'SC', 'RS',
       'PE', 'CE', 'MA', 'PA', 'SP', 'PR', 'TO', 'RO', 'MG', 'BA', 'RJ',
       'SE', 'DF', 'AL', 'RN', 'PI', nan], dtype=object)

In [25]:
df['sexo'].unique()

array([2, 1, 0])

In [26]:
df[df['peso'] <= 0]

,ano,sigla_uf,local_nascimento,data_nascimento,hora_nascimento,sexo,peso,raca_cor,id_anomalia,codigo_anomalia,...,quantidade_filhos_mortos,idade_mae,escolaridade_mae,estado_civil_mae,ocupacao_mae,raca_cor_mae,gestacoes_ant,quantidade_parto_normal,quantidade_parto_cesareo,idade_pai
4906756,2015,AM,3,2015-03-04,10:30:00,2,0.0,5.0,2.0,NaN,...,0.0,29.0,2.0,1.0,NaN,5.0,1.0,1.0,0.0,43.0
8184201,2015,AP,5,2015-09-12,04:00:00,1,0.0,5.0,2.0,NaN,...,1.0,26.0,2.0,5.0,NaN,5.0,5.0,4.0,0.0,29.0
16468008,2015,AM,5,2015-03-02,17:00:00,2,0.0,5.0,2.0,NaN,...,0.0,21.0,3.0,1.0,622005.0,5.0,4.0,4.0,0.0,30.0


In [27]:
df = df[df['peso'] > 0]

In [28]:
df['tipo_parto'].unique()

array([2., 1., 9.])

In [29]:
df['tipo_gravidez'].unique()

array([1., 2., 9., 3.])

In [30]:
np.sort(df['idade_mae'].unique())

array([ 8.        ,  9.        , 10.        , 11.        , 12.        ,
       13.        , 14.        , 15.        , 16.        , 17.        ,
       18.        , 19.        , 20.        , 21.        , 22.        ,
       23.        , 24.        , 25.        , 26.        , 26.83580149,
       27.        , 28.        , 29.        , 30.        , 31.        ,
       32.        , 33.        , 34.        , 35.        , 36.        ,
       37.        , 38.        , 39.        , 40.        , 41.        ,
       42.        , 43.        , 44.        , 45.        , 46.        ,
       47.        , 48.        , 49.        , 50.        , 51.        ,
       52.        , 53.        , 54.        , 55.        , 56.        ,
       57.        , 58.        , 59.        , 60.        , 61.        ,
       62.        , 63.        , 64.        , 65.        , 99.        ])

In [31]:
df = df[df['idade_mae'] < 99]

Detectamos valores inválidos para as colunas `peso` (3 linhas) e `idade_mae` (13 linhas). Esses registros foram removidos.

### 5. Transformando tipos de dados

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19998679 entries, 0 to 2677100
Data columns (total 27 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   ano                       int64  
 1   sigla_uf                  object 
 2   local_nascimento          int64  
 3   data_nascimento           object 
 4   hora_nascimento           object 
 5   sexo                      int64  
 6   peso                      float64
 7   raca_cor                  float64
 8   id_anomalia               float64
 9   codigo_anomalia           object 
 10  semana_gestacao           float64
 11  gestacao_agr              float64
 12  tipo_gravidez             float64
 13  tipo_parto                float64
 14  pre_natal                 float64
 15  pre_natal_agr             float64
 16  quantidade_filhos_vivos   float64
 17  quantidade_filhos_mortos  float64
 18  idade_mae                 float64
 19  escolaridade_mae          float64
 20  estado_civil_mae          fl

In [50]:
# Convertendo 'object' para 'date'
import datetime

df['data_nascimento'] = pd.to_datetime(df['data_nascimento'], format='%Y-%m-%d', errors='coerce').fillna(
    pd.to_datetime(df['data_nascimento'], format='%d%m%Y', errors='coerce')
)

df['data_nascimento']

0         2016-01-25
1         2016-02-01
2         2016-02-07
3         2016-04-11
4         2016-04-15
             ...    
2677096   2021-09-19
2677097   2021-10-30
2677098   2021-09-21
2677099   2021-02-11
2677100   2021-12-08
Name: data_nascimento, Length: 19998679, dtype: datetime64[ns]

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19998679 entries, 0 to 2677100
Data columns (total 27 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   ano                       int64         
 1   sigla_uf                  object        
 2   local_nascimento          int64         
 3   data_nascimento           datetime64[ns]
 4   hora_nascimento           object        
 5   sexo                      int64         
 6   peso                      float64       
 7   raca_cor                  float64       
 8   id_anomalia               float64       
 9   codigo_anomalia           object        
 10  semana_gestacao           float64       
 11  gestacao_agr              float64       
 12  tipo_gravidez             float64       
 13  tipo_parto                float64       
 14  pre_natal                 float64       
 15  pre_natal_agr             float64       
 16  quantidade_filhos_vivos   float64       
 17  quantidade_f

### 6. Salvando arquivo limpo

In [77]:
df.to_csv('../datasets/nascidos_vivos_limpo.csv', index=False)